In [ ]:
!pip install yfinance
!pip install pytickersymbols

In [ ]:
import os
import requests 
from io import StringIO
import yfinance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pytickersymbols import PyTickerSymbols
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
from time import sleep
import plotly.graph_objects as go
import random

In [ ]:
stock_data = PyTickerSymbols()

nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index().symbol.to_list()

sp_500=pd.DataFrame(stock_data.get_stocks_by_index('S&P 500'))
sp_500 = sp_500[sp_500.symbol != 'SSE']
sp_500 = sp_500[sp_500.symbol != 'RTN']
sp_500 = sp_500[sp_500.symbol != 'CXO']
sp_500 = sp_500[sp_500.symbol != 'JEC']
sp_500 = sp_500[sp_500.symbol != 'SPY']
sp_500 = sp_500[sp_500.symbol != 'TIF'].reset_index().symbol.to_list()

stocks = nasdaq_100 + sp_500 #+ sp400

stocks = set(stocks) #Convierte la lista a Set, que lo único que hace es quitar elementos repetidos
stocks = list(stocks) #Reconvertir el set a list (para poder iterar sobre ella)

In [ ]:
#stocks = random.sample(stocks, 1)
#stocks = ['SPY']

In [ ]:
len(stocks)

514

In [ ]:
df_plot = pd.DataFrame()
for t in tqdm(range(len(stocks))):
  #OPCION PARA DATOS CON YFINANCE ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  data = yfinance.download(stocks[t], period='1y')#[['Open', 'High', 'Low', 'Close']]
  data['Ticker'] = stocks[t]
  data['Gap'] = np.nan
  data['Gap&Go'] = np.nan
  data['Red'] = np.nan

  for i in range(1,len(data)):
    gap = (data['Open'][i] - data['Close'][i-1]) / data['Close'][i-1]
    data['Gap'][i] = gap
    gapandgo = 0
    counter = 1
    while (data['Open'][i] < data['Low'][i-counter]) and (data['Close'][i-counter] >=  data['Open'][i-counter]) and (i != counter):
      gapandgo += 1
      counter += 1
    data['Gap&Go'][i] = gapandgo
    if data['Gap&Go'][i] > 0 and (data['Close'][i] < data['Open'][i]):
      data['Red'][i] = True

  df_plot = pd.concat([df_plot,data])

In [ ]:
df_plot

,Open,High,Low,Close,Adj Close,Volume,Ticker,Gap,Gap&Go,Red
Date,,,,,,,,,,
2020-05-19,125.110001,125.970001,122.769997,123.169998,121.232185,7914900.0,TGT,NaN,NaN,NaN
2020-05-20,124.089996,124.910004,119.290001,119.629997,117.747879,12897000.0,TGT,0.007469,0.0,NaN
2020-05-21,119.629997,120.370003,117.639999,118.680000,116.812836,7620700.0,TGT,0.000000,0.0,NaN
2020-05-22,118.830002,119.360001,117.180000,117.489998,115.641548,4902900.0,TGT,0.001264,0.0,NaN
2020-05-26,118.800003,119.959999,114.809998,115.120003,113.308838,7274600.0,TGT,0.011150,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2021-04-09,177.360001,177.360001,177.210007,177.210007,177.210007,457014.0,VAR,0.000790,0.0,NaN
2021-04-12,177.250000,177.300003,176.990005,177.119995,177.119995,713531.0,VAR,0.000226,0.0,NaN
2021-04-13,177.100006,177.259995,177.100006,177.110001,177.110001,319875.0,VAR,-0.000113,0.0,NaN


In [ ]:
#df_plot.sort_values(by='Date', ascending=False)

## FILTRO PARA ESTRATEGIA

In [ ]:
filtro = df_plot[(df_plot['Gap&Go'] > 0) &
                 (df_plot['Gap'] < -.04)]# & 
                 #(df_plot['Open'] > 10) & 
                 #(df_plot['Volume'] > 750000)]
#filtro['red_day'] = (filtro['Close'] < filtro['Open']) * 1

In [ ]:
filtro

,Open,High,Low,Close,Adj Close,Volume,Ticker,Gap,Gap&Go,Red
Date,,,,,,,,,,
2021-04-22,58.270000,60.060001,57.750000,58.330002,58.330002,12097900.0,LVS,-0.040981,1.0,NaN
2020-07-28,65.260002,68.480003,62.205002,62.419998,61.608467,3554400.0,LW,-0.074326,1.0,1.0
2020-06-15,12.620000,13.800000,12.510000,13.620000,13.249839,6629300.0,KIM,-0.082849,1.0,NaN
2021-03-17,103.760002,105.089996,102.800003,105.040001,103.778206,19445600.0,ABBV,-0.063876,3.0,NaN
2020-06-05,45.119999,46.880001,44.410000,45.900002,45.900002,8244100.0,PTON,-0.057644,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-06-09,79.839996,80.540001,77.330002,79.209999,76.744446,1110100.0,HAS,-0.048618,1.0,1.0
2020-10-26,87.019997,87.900002,82.250000,83.400002,81.560669,3951200.0,HAS,-0.054130,1.0,1.0
2021-01-04,352.779999,385.000000,350.010010,362.390015,362.390015,1452300.0,TDY,-0.100005,2.0,NaN


In [ ]:
filtro['Red'].value_counts()

1.0    107
Name: Red, dtype: int64

In [ ]:
df_plot.loc['2020-12-08':'2020-12-11']

,Open,High,Low,Close,Adj Close,Volume,Ticker,Gap,Gap&Go
Date,,,,,,,,,
2020-12-08,39.279999,39.950001,39.180000,39.520000,38.942562,3322600,AIG,-0.012817,0.0
2020-12-09,39.910000,40.290001,39.560001,39.939999,39.356422,5388400,AIG,0.009868,0.0
2020-12-10,39.279999,39.939999,39.060001,39.709999,39.129784,4716300,AIG,-0.016525,1.0
2020-12-11,38.830002,39.169998,38.560001,38.790001,38.533749,5299000,AIG,-0.022161,3.0


In [ ]:
len(filtro)

34

In [ ]:
filtro = filtro.sample(500)
len(filtro)

500

# VERSION ALPHAVANTAGE (UN AÑO, MAX 500 REQ)

In [ ]:
#Se crea la lista y la división de las fechas desde hoy hasta donde se espcifique atrás (una división son 30 días):

datelist = pd.date_range(end=datetime.today(), periods=360).strftime('%Y-%m-%d').tolist()
split1, split2, split3, split4, split5, split6, split7, split8, split9, split10, split11, split12, = np.array_split(datelist, 12)

#Se asignan las stocks que entran en el Filtro a cada división
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12 = [], [], [],[],[],[],[],[],[],[],[],[]
for i in range(len(filtro)):
  dia = filtro.index[i].strftime('%Y-%m-%d')
  if dia in split1:
    df1.append([dia,filtro['Ticker'][i]])
  elif dia in split2:
    df2.append([dia,filtro['Ticker'][i]])
  elif dia in split3:
    df3.append([dia,filtro['Ticker'][i]])
  elif dia in split4:
    df4.append([dia,filtro['Ticker'][i]])
  elif dia in split5:
    df5.append([dia,filtro['Ticker'][i]])
  elif dia in split6:
    df6.append([dia,filtro['Ticker'][i]])
  elif dia in split7:
    df7.append([dia,filtro['Ticker'][i]])
  elif dia in split8:
    df8.append([dia,filtro['Ticker'][i]])
  elif dia in split9:
    df9.append([dia,filtro['Ticker'][i]])
  elif dia in split10:
    df10.append([dia,filtro['Ticker'][i]])
  elif dia in split11:
    df11.append([dia,filtro['Ticker'][i]])
  elif dia in split12:
    df12.append([dia,filtro['Ticker'][i]])

df_total = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12]
slices = ['year1month12','year1month11','year1month10','year1month9','year1month8','year1month7',
          'year1month6','year1month5','year1month4','year1month3','year1month2','year1month1']

# VERSION ALPHAVANTAGE (6 MESES, MAX 500 REQ)

In [ ]:
# #VERSION PARA 6 MESES

# datelist = pd.date_range(end=datetime.today(), periods=180).strftime('%Y-%m-%d').tolist()
# split1, split2, split3, split4, split5, split6 = np.array_split(datelist, 6)

# #Se asignan las stocks que entran en el Filtro a cada división
# df1, df2, df3, df4, df5, df6 = [],[],[],[],[],[]
# for i in range(len(filtro)):
#   dia = filtro.index[i].strftime('%Y-%m-%d')
#   if dia in split1:
#     df1.append([dia,filtro['Ticker'][i]])
#   elif dia in split2:
#     df2.append([dia,filtro['Ticker'][i]])
#   elif dia in split3:
#     df3.append([dia,filtro['Ticker'][i]])
#   elif dia in split4:
#     df4.append([dia,filtro['Ticker'][i]])
#   elif dia in split5:
#     df5.append([dia,filtro['Ticker'][i]])
#   elif dia in split6:
#     df6.append([dia,filtro['Ticker'][i]])

# df_total = [df1, df2, df3, df4, df5, df6]
# slices = ['year1month6','year1month5','year1month4','year1month3','year1month2','year1month1']

In [ ]:
os.environ["ALPHA_VANTAGE_KEY"]="60MP74RUOD4MUHKI"
key=os.environ["ALPHA_VANTAGE_KEY"]
alphav_url="https://www.alphavantage.co/query"

In [ ]:
#Loop para generar las stocks del filtro en timeframe de 2 min (por cada división)
target1, green1, red1, stop1, target2, green2, red2, stop2, target3, green3, red3, stop3, = [], [], [], [], [], [], [], [], [], [], [], [],

ix = 0
for i in tqdm(range(len(slices))):
  if len(df_total[i]) > 0:
    for j in tqdm(range(len(df_total[i]))):
      params={
        "apikey":key,
        "function":"TIME_SERIES_INTRADAY_EXTENDED",
        'interval': '1min',
        "symbol": df_total[i][j][1],
        "slice":slices[i],
        "datatype":"csv",
        "outputsize":"compact"
        }
      
      r=requests.get(alphav_url,params=params)
      example=pd.read_csv(StringIO(r.text))
      
      example['time']= pd.to_datetime(example['time'])
      example=(example.set_index("time")).sort_values(by='time', ascending=True)
      example = example.loc[df_total[i][j][0] + ' ' + '09:30:00' : df_total[i][j][0] + ' ' + '16:00:00']
      ohlc_dict = {'open':'first', 'high':'max', 'low':'min', 'close':'last', 'volume':'sum'}
      example = example.resample('2T', closed='left', label='left').apply(ohlc_dict)

      #Entry si el segundo candle cae debajo del primero
      if len(example) > 0 and example.iloc[1]['low'] < example.iloc[0]['low']:
        stop = max(example.iloc[1]['high'], example.iloc[1]['high']) + .01 #Stop = valor máximo de los Highs del primer y segundo candle
        entry = example.iloc[0]['low'] - .01 #Entry = Low del primer candle

        for k1 in range(1,len(example)):
          target = entry - (1 * (stop-entry)) #Target = el número de risks por el monto al stop
          if example.iloc[k1]['high'] > stop:
            stop1.append(df_total[i][j][1])
            break
          elif example.iloc[k1]['low'] < target:
            target1.append(df_total[i][j][1])
            break
          elif k1 == len(example):
            if example.iloc[k1]['close'] <= entry:
              green1.append(df_total[i][j][1])
              break
            else:
              red1.append(df_total[i][j][1])
              break

        for k2 in range(1,len(example)):
          target = entry - (2 * (stop-entry)) #Target = el número de risks por el monto al stop
          if example.iloc[k2]['high'] > stop:
            stop2.append(df_total[i][j][1])
            break
          elif example.iloc[k2]['low'] < target:
            target2.append(df_total[i][j][1])
            break
          elif k2 == len(example):
            if example.iloc[k2]['close'] <= entry:
              green2.append(df_total[i][j][1])
              break
            else:
              red2.append(df_total[i][j][1])
              break

        for k3 in range(1,len(example)):
          target = entry - (3 * (stop-entry)) #Target = el número de risks por el monto al stop
          if example.iloc[k3]['high'] > stop:
            stop3.append(df_total[i][j][1])
            break
          elif example.iloc[k3]['low'] < target:
            target3.append(df_total[i][j][1])
            break
          elif k3 == len(example):
            if example.iloc[k3]['close'] <= entry:
              green3.append(df_total[i][j][1])
              break
            else:
              red3.append(df_total[i][j][1])
              break
      #print(ticker_df.head(3))
      # fig = go.Figure(data=[go.Candlestick(open=ticker_df.open, high=ticker_df.high,low=ticker_df.low, close=ticker_df.close)])
      # fig.update_layout(xaxis_rangeslider_visible=False)
      # fig.update_layout(title=df_total[i][j][1] + ' ' + df_total[i][j][0]) #+ ' ' + str(filtro.Gap[i]))
      # fig.show()

      ix +=1
      if ix % 4==0:
        sleep(30)

In [ ]:
print(len(target1))
print(len(green1))
print(len(red1))
print(len(stop1))
print('Rs totales con Risk de 1:', len(target1)-len(stop1))
print(50*'-')
print(len(target2))
print(len(green2))
print(len(red2))
print(len(stop2))
print('Rs totales con Risk de 2:', (2*len(target2))-len(stop2))
print(50*'-')
print(len(target3))
print(len(green3))
print(len(red3))
print(len(stop3))
print('Rs totales con Risk de 3:', (3*len(target3))-len(stop3))

71
0
0
55
Rs totales con Risk de 1: 16
--------------------------------------------------
49
0
0
74
Rs totales con Risk de 2: 24
--------------------------------------------------
38
0
0
80
Rs totales con Risk de 3: 34


In [ ]:
# for i in tqdm(range(len(green_stocks))):

#   example = yfinance.download(green_stocks[i], start=green_stocks[i].strftime('%Y-%m-%d'), # - timedelta(days=1))
#                             end=(green_stocks[i] + timedelta(days=1)).strftime('%Y-%m-%d'), interval='2m')[['Open', 'High', 'Low', 'Close']]

#   fig = go.Figure(data=[go.Candlestick(open=example.Open, high=example.High,low=example.Low, close=example.Close)])
#   fig.update_layout(xaxis_rangeslider_visible=False)
#   fig.update_layout(title=green_stocks[i])
#   fig.show()

In [ ]:
# #PARA REVISION INDIVIDUAL POR STOCK
# params={
#   "apikey":key,
#   "function":"TIME_SERIES_INTRADAY_EXTENDED",
#   'interval': '1min',
#   "symbol": 'SPY',#df_total[i][j][1],
#   "slice":slices[i],
#   "datatype":"csv",
#   "outputsize":"compact"
#   }

# r=requests.get(alphav_url,params=params)
# ticker_df=pd.read_csv(StringIO(r.text))

# #ticker_df=(ticker_df.set_index("time")).sort_values(by='time', ascending=True)
# #ticker_df = ticker_df.loc[df_total[i][j][0] + ' ' + '09:30:00' : df_total[i][j][0] + ' ' + '16:00:00']
# print(ticker_df.head())

# VERSION YFINANCE (SOLO 40 DIAS)

In [ ]:
# risk = 1
# green_stocks = []
# red_stocks = []
# for i in tqdm(range(len(filtro))):

#   example = yfinance.download(filtro.Ticker[i], start=filtro.index[i].strftime('%Y-%m-%d'), # - timedelta(days=1))
#                             end=(filtro.index[i] + timedelta(days=1)).strftime('%Y-%m-%d'), interval='2m')[['Open', 'High', 'Low', 'Close']]
#   #Entry si el segundo candle cae debajo del primero
#   if len(example) > 0 and example.iloc[1]['Low'] < example.iloc[0]['Low']:
#     stop = max(example.iloc[1]['High'], example.iloc[1]['High']) + .01 #Stop = valor máximo de los Highs del primer y segundo candle
#     entry = example.iloc[0]['Low'] - .01 #Entry = Low del primer candle
#     target = entry - (risk * (stop-entry)) #Target = el número de risks por el monto al stop
#     for j in range(1,len(example)):
#       if example.iloc[j]['High'] > stop:
#         red_stocks.append(filtro.Ticker[i])
#         break
#       elif example.iloc[j]['Low'] < target:
#         green_stocks.append(filtro.Ticker[i])
#         break

#   # fig = go.Figure(data=[go.Candlestick(open=example.Open, high=example.High,low=example.Low, close=example.Close)])
#   # fig.update_layout(xaxis_rangeslider_visible=False)
#   # fig.update_layout(title=filtro.Ticker[i] + ' ' + filtro.index[i].strftime('%Y-%m-%d') + ' ' + str(filtro.Gap[i]))
#   # fig.show()

In [ ]:
# print(green_stocks)
# print(red_stocks)
# print(len(green_stocks))
# print(len(red_stocks))